## 20.3 LDA的吉布斯抽样算法

LDA的学习（参数估计）通常只能用近似方法求解。常用的近似方法有吉布斯抽样（Gibbs sampling）和变分推理（variational inference）。

### 20.3.1 基本想法

给定:  

文本（单词序列）的集合$D=\left\{\mathbf{w}_{1}, \cdots, \mathbf{w}_{m}, \cdots, \mathbf{w}_{M}\right\}$，其中$\mathbf{w}_{m}=\left(w_{m 1}, \cdots, w_{m n}, \cdots, w_{m N_{m}}\right)$  
以$\mathbf{w}$表示文本集合的单词序列，即$\mathbf{w}=\left(w_{11}, w_{12}, \cdots, w_{1 N_{1}}, w_{21}, w_{22}, \cdots, w_{2 N_{2}}, \cdots, w_{M 1}, w_{M 2}, \cdots, w_{MN_{M}}\right)$  

超参数$~\alpha~$和$~\beta~$已知。  

&nbsp;

目标是推断：  

（1）话题序列的集合 $\mathbf{z}=\left\{\mathbf{z}_{1}, \cdots, \mathbf{z}_{m}, \cdots, \mathbf{z}_{M}\right\}$的后验概率分布，其中$\mathbf{z}_{m}$是第$~m~$个文本的话题序列，$\mathbf{z}_{m}=\left(z_{m 1}, \cdots, z_{m n}, \cdots, z_{m N_{m}}\right)$； 

（2）参数$\theta=\left\{\theta_{1}, \cdots, \theta_{m}, \cdots, \theta_{M}\right\}$，其中$\theta_{m}$是文本$\mathbf{w}_{m}$的话题分布的参数，$\theta_{m}=\left(\theta_{m 1}, \theta_{m 2}, \cdots, \theta_{m K}\right)$；  

（3）参数$\varphi=\left\{\varphi_{1}, \cdots, \varphi_{k}, \cdots, \varphi_{K}\right\}$，其中$\varphi_{k}$是话题$z_{k}$的单词分布的参数，$\varphi_{k}=\left(\varphi_{k 1}, \varphi_{k 2}, \cdots, \varphi_{k V}\right)$。

也就是说对联合概率分布$~p(\mathbf{w}, \mathbf{z}, \theta, \varphi | \alpha, \beta)~$进行估计，其中$~\mathbf{w}~$是观测变量，而$~\mathbf{z}, \theta, \varphi~$是隐变量。

&nbsp;

<font color=red>LDA模型的学习通常采用**收缩的吉布斯抽样（collapsed Gibbs sampling）方法**</font>: （<font color=red>????</font>）
1. <font color=blue>通过对隐变量$~\theta,\varphi~$积分，得到边缘概率分布$~p(\mathbf{w}, \mathbf{z} | \alpha, \beta)~$</font>；

2. <font color=blue>对后验概率分布$~\color{red}{p( \mathbf{z} |\mathbf{w}, \alpha, \beta)}~$进行吉布斯抽样，得到分布的样本集合</font>；

3. <font color=blue>再利用这个样本集合对参数$~\theta~$和$~\varphi~$进行估计，最终得到LDA模型$~p(\mathbf{w}, \mathbf{z}, \theta, \varphi | \alpha, \beta)~$的所有参数估计</font>。

### 20.3.2 算法的主要部分

关键是对后验概率分布$~\color{red}{p( \mathbf{z} |\mathbf{w}, \alpha, \beta)}~$的吉布斯抽样。  

<img src='20.3_后验概率分布.png' style='zoom:40%'/>

#### 1. 抽样分布的表达式

后验概率分布  

$$
p(\mathbf{z} | \mathbf{w}, \alpha, \beta)=\frac{p(\mathbf{w}, \mathbf{z} | \alpha, \beta)}{p(\mathbf{w} | \alpha, \beta)} \propto p(\mathbf{w}, \mathbf{z} | \alpha, \beta) \tag{20.20}
$$

对联合分布$p(\mathbf{w}, \mathbf{z} | \alpha, \beta)$进行分解

$$
p(\mathbf{w}, \mathbf{z} | \alpha, \beta)=p(\mathbf{w} | \mathbf{z}, \alpha, \beta) p(\mathbf{z} | \alpha, \beta)=p(\mathbf{w} | \mathbf{z}, \beta) p(\mathbf{z} | \alpha) \tag{20.21}
$$

<font color=blue>对第一个因子$p(\mathbf{w} | \mathbf{z}, \beta)$</font>，首先: 

$$
p(\mathbf{w} \mid \mathbf{z}, \varphi)=\prod_{k=1}^{K} \prod_{v=1}^{V} \varphi_{k v}^{n_{k v}}
$$

其中，$~\varphi_{k v}~$是第$~k~$个话题生成第$~v~$个单词的概率，$~n_{k v}~$是数据中第$~k~$个话题生成第$~v~$个单词的次数，于是: 

$$
\begin{eqnarray}
p(\mathbf{w} | \mathbf{z}, \beta) &=& \int p(\mathbf{w} | \mathbf{z}, \varphi) p(\varphi | \beta) \mathrm{~d} \varphi \\
&=& \int \prod_{k=1}^{K} \frac{1}{\mathrm{~B}(\beta)} \prod_{v=1}^{V} \varphi_{k v}^{n_{k v}+\beta_{v}-1} \mathrm{~d} \varphi \\
&=& \prod_{k=1}^{K} \frac{1}{\mathrm{~B}(\beta)} \int \prod_{v=1}^{V} \varphi_{k v}^{n_{k v}+\beta_{v}-1} \mathrm{~d} \varphi \\
&=& \prod_{k=1}^{K} \frac{\mathrm{B}\left(n_{k}+\beta\right)}{\mathrm{B}(\beta)} \tag{20.23}
\end{eqnarray}
$$

其中，$n_{k}=\left\{n_{k 1}, n_{k 2}, \cdots, n_{k V}\right\}$，$\beta=\left(\beta_{1}, \beta_{2}, \cdots, \beta_{V}\right)$

<font color=blue>对第二个因子$p(\mathbf{z} | \alpha)$</font>，首先: 

$$
p(\mathbf{z} | \theta)=\prod_{m=1}^{M} \prod_{k=1}^{K} \theta_{m k}^{n_{m k}} \tag{20.24}
$$

其中，$~\theta_{m k}~$是第$~m~$个文本生成第$~k~$个话题的概率，$~n_{m k}~$是数据中第$~m~$个文本生成第$~k~$个话题的次数。于是: 

$$
\begin{eqnarray}
p(\mathbf{z} \mid \alpha) &=& \int p(\mathbf{z} \mid \theta) p(\theta \mid \alpha) \mathrm{~d} \theta \\
&=&\int \prod_{m=1}^{M} \frac{1}{\mathrm{~B}(\alpha)} \prod_{k=1}^{K} \theta_{m k}^{n_{m k}+\alpha_{k}-1} \mathrm{~d} \theta \\
&=& \prod_{m=1}^{M} \frac{1}{\mathrm{~B}(\alpha)} \int \prod_{k=1}^{K} \theta_{m k}^{n_{m k}+\alpha_{k}-1} \mathrm{~d} \theta \\
&=&\prod_{m=1}^{M} \frac{\mathrm{B}\left(n_{m}+\alpha\right)}{\mathrm{B}(\alpha)}
\end{eqnarray} \tag{20.25}
$$

其中，$n_{m}=\left\{n_{m 1}, n_{m 2}, \cdots, n_{m K}\right\}$，$\alpha=\left(\alpha_{1}, \alpha_{2}, \cdots, \alpha_{K}\right)$ 

由（20.23）和（20.25）式可得

$$
p(\mathbf{w}, \mathbf{z} | \alpha, \beta) = \prod_{k=1}^{K} \frac{\mathrm{B}\left(n_{k}+\beta\right)}{\mathrm{B}(\beta)} \cdot \prod_{m=1}^{M} \frac{\mathrm{B}\left(n_{m}+\alpha\right)}{\mathrm{B}(\alpha)} \tag{20.26}
$$

于是得到收缩的吉布斯抽样分布公式:  

$$
p( \mathbf{z} |\mathbf{w}, \alpha, \beta) \propto  \prod_{k=1}^{K} \frac{\mathrm{B}\left(n_{k}+\beta\right)}{\mathrm{B}(\beta)} \cdot \prod_{m=1}^{M} \frac{\mathrm{B}\left(n_{m}+\alpha\right)}{\mathrm{B}(\alpha)} \tag{20.27}
$$